In [39]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openpyxl
import re
from selenium import webdriver
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from datetime import datetime

## 2018년 성공한 테크 프로젝트들 가져오기

In [2]:
data = pd.read_excel('master_data.xlsx')

In [4]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
data.head()

,url,메이커,펀딩시작날짜,펀딩마감날짜,펀딩시작요일,목표금액,서포터수,좋아요수,글업데이트수,댓글수,...,패션·잡화,푸드,홈리빙,분기,와디즈기사개수,크라우드펀딩기사개수,PER,PBR,ROE,EBIT
0,https://www.wadiz.kr/web/campaign/detail/17233,건강한연구소,2018-06-27,2018-08-01,Wed,1000000,318,193,6,73,...,0,1,0,18Q1적용,30,269,23.014000,1.318333,9.276333,116.833333
1,https://www.wadiz.kr/web/campaign/detail/19909,카페 펠리스,2018-06-27,2018-07-31,Wed,1000000,46,24,5,17,...,0,1,0,18Q1적용,30,269,23.014000,1.318333,9.276333,116.833333
2,https://www.wadiz.kr/web/campaign/detail/20844,Future Food Lab,2018-06-27,2018-07-22,Wed,1000000,559,168,6,116,...,0,1,0,18Q1적용,30,269,23.014000,1.318333,9.276333,116.833333
3,https://www.wadiz.kr/web/campaign/detail/21456,농부대첩,2018-06-27,2018-07-09,Wed,1000000,543,232,17,136,...,0,1,0,18Q1적용,30,269,23.014000,1.318333,9.276333,116.833333
4,https://www.wadiz.kr/web/campaign/detail/18871,스튜디오와츠,2018-06-27,2018-07-31,Wed,500000,3,10,1,0,...,0,0,0,18Q1적용,30,269,12.680769,1.122308,3.486923,96.000000


In [7]:
data[data['펀딩시작날짜']=='2018-01']

,url,메이커,펀딩시작날짜,펀딩마감날짜,펀딩시작요일,목표금액,서포터수,좋아요수,글업데이트수,댓글수,...,패션·잡화,푸드,홈리빙,분기,와디즈기사개수,크라우드펀딩기사개수,PER,PBR,ROE,EBIT


In [8]:
data.set_index('펀딩시작날짜', inplace=True)

In [14]:
tech_data2018 = data.loc['2018']

In [17]:
tech_data2018 = tech_data2018[tech_data2018.카테고리 == '테크·가전']

In [21]:
tech_data2018.sort_index(inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
tech_data2018.columns

Index(['펀딩시작날짜', 'url', '메이커', '펀딩마감날짜', '펀딩시작요일', '목표금액', '서포터수', '좋아요수',
       '글업데이트수', '댓글수', '리워드종류수', '이미지수', '비디오수', '배송시작날짜', '마감배송차이',
       '인스타존재여부', '인스타팔로워수', '와디즈팔로워수', '과거프로젝트수', '과거성공프로젝트수', '앵콜펀딩여부',
       '음절수', '단어수', '문장수', 'Strong', 'Under', '가독성', '가독성2', '제목단어수',
       '펀딩성공여부', '달성률', '펀딩시작월', '소비자물가', '근원물가', '생활물가', '실질GDP성장률', '제목',
       '달성액', '요약글', '카테고리', '디자인소품', '반려동물', '뷰티', '스포츠·모빌리티', '여행·레저',
       '테크·가전', '패션·잡화', '푸드', '홈리빙', '분기', '와디즈기사개수', '크라우드펀딩기사개수', 'PER',
       'PBR', 'ROE', 'EBIT'],
      dtype='object')

In [24]:
tech_data2018 = tech_data2018[tech_data2018.펀딩성공여부 == 1]

In [28]:
tech_data2018 = tech_data2018.reset_index()

In [31]:
tech_data2018 = tech_data2018[['url', '메이커', '제목', '펀딩시작날짜', '펀딩마감날짜', '카테고리', '펀딩성공여부']]

In [36]:
tech_data2018

,url,메이커,제목,펀딩시작날짜,펀딩마감날짜,카테고리,펀딩성공여부
0,https://www.wadiz.kr/web/campaign/detail/16418,엠케이테크놀로지,스마트폰 사진이 예술작품으로 변하다 : 레볼캠,2018-01-04,2018-02-04,테크·가전,1
1,https://www.wadiz.kr/web/campaign/detail/16272,반디통신기술(주),"일상을 스마트하게, IoT 전등스위치",2018-01-08,2018-02-22,테크·가전,1
2,https://www.wadiz.kr/web/campaign/detail/14973,(주)스토리포유,"진정한 마음의 평화를 가져다 줄 작은 동반자, ""에스체인""",2018-01-15,2018-02-18,테크·가전,1
3,https://www.wadiz.kr/web/campaign/detail/16853,(주)토브넷,"""이런 CCTV는 처음이지? 나야 나! 토브넷캠!""",2018-01-19,2018-02-18,테크·가전,1
4,https://www.wadiz.kr/web/campaign/detail/15982,오정숙,굿바이~ 문콕! 차량용 프리미엄 도어가드 GP-01,2018-01-23,2018-03-07,테크·가전,1
5,https://www.wadiz.kr/web/campaign/detail/16617,신텍주식회사,3가지 기술을 하나로! 블루투스 캐치벨!,2018-01-24,2018-03-11,테크·가전,1
6,https://www.wadiz.kr/web/campaign/detail/16213,씨익연구소,"이야기를 전하는 나만의 공기청정기, 에어챗 공기청정기",2018-01-24,2018-03-11,테크·가전,1
7,https://www.wadiz.kr/web/campaign/detail/14869,(주)야베스 인터내셔널,하이브리드 스마트 워치 'ZeTime 지타임',2018-02-06,2018-03-11,테크·가전,1
8,https://www.wadiz.kr/web/campaign/detail/16950,신지모루,어디에나 딱! 편리하게 부착하는 차량용 거치대 신지모루 볼리에어,2018-02-20,2018-03-18,테크·가전,1
9,https://www.wadiz.kr/web/campaign/detail/16964,골드래빗(주),누구나 원하는 로봇을 만들 수 있습니다. 마운지 로봇!,2018-02-28,2018-04-13,테크·가전,1


In [42]:
tech_data2018.iloc[1]['펀딩마감날짜'].strftime('%Y.%m.%d')

'2018.02.22'

## 성공한 테크 프로젝트들 프로젝트명 가져오기

In [47]:
# 엑셀 파일 만들기
wb = openpyxl.Workbook()
sheet = wb.active

sheet.append(['url', '프로젝트명'])

# 크롬 드라이버(창 띄우기 않고 데이터 가져오게 설정)
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver', options=options)

for i in list(tech_data2018.url):
    driver.get(i)
    project_name = driver.find_element_by_css_selector('p.title-info strong').text.replace('#', '')
    
    print(project_name)
    sheet.append([i, project_name])
    
wb.save('tech_data_project_names_2018.xlsx')

레볼캠
WiFi전등스위치
에스체인프로
말하는전구CCTV
굿바이문콕
위치기반블루투스
에어챗공기청정기
스마트워치지타임
차량용휴대폰거치대
놀라운로봇마운지
오클린원음파칫솔
스마트플러그
시력보호필름
유튜브크리에이터
스마트버튼팝팝
무선충전마우스패드
무선충전거치대
기능칫솔콧숨테이프
지라이더미니선풍기
엣지E6플러스
스마트주차번호패드
카딱스마트재떨이
방부제제로가글
탈취항균음쓰레기통
몸에좋은물림피오
모기퇴치기
핸디형미니선풍기
이노블프라이팬
수소수텀블러
미세먼지측정기
위너스멘토
화면분할모니터
환기청정기
착붙여쓰는배터리
웹카메라커버
우든티비나무티비
타자기키보드
그립감보조신지그립
카딱스마트충전기
최소형오디오
미니헤드선풍기
스마트안경시선비콘
스마트주방저울
초소형유해물질센서
인공지능로봇
원목미니피씨
영유아용공기청정기
골전도헤드폰
Pet자동사료배식기
창문형공기청정기
극강의근육강화
공기청정기에어링
차량용스마트디퓨저
충전식수험생시계
무한확장명품스피커
듀얼사운드선글라스
두피열잡는두피쿨
젠틀사운드HD
누구나쓰는광케이블
책상위의공기청정기
대쉬크랩에어시리즈
에이치에어워셔
모듈무선충전기
다오손풍기
스마트초인종
매직올인원스마트링
건강한외출동반자
VH80
블루투스우퍼이어셋
저주파마사지기
세계최초물청소기
세계최초듀얼공유기
수돗물을자화수로
나만의청정기에어챗
닥터삭스
무선블루투스이어폰
타자기키보드
여행용진공포장기
나무헤드폰
루프트에어컨필터
고속무선충전기UMU
부엉이USB클릭탭
유무선고속충전패드
변신써큘레이터
차량용무선청소기
100W고속충전기
무선신발살균건조기
아이폰백업큐비
고속무선충전기
러기드외장하드
칫솔질의혁명
차량용텀블러재떨이
만능스쿠버장비
무선믹서기텀블러
완벽한충전기댕글
차랑용공기청정기
힐링팟가습기
수면헤드폰
차량무선충전거치대
리버스보온병
지문인증장치
나만의EMS홈트레이
캐시비스마트밴드
차모아목쿠션안마기
무선충전Qi독
타자기키보드
에어프라이어
콜드월렛
퀸메이드무선청소기
차량용충전거치대
PaMu
생리컵소독기
WELLVENT
레쥬빔텀블러
국내산공기청정기
릴케이블무선충전기
졸음방지산소뿜뿜
붙이는보조배터리
올인

## 데이터프레임 합치기

In [48]:
project_names = pd.read_excel('tech_data_project_names_2018.xlsx')

In [50]:
tech_data2018 = pd.merge(tech_data2018, project_names)

In [51]:
tech_data2018

,url,메이커,제목,펀딩시작날짜,펀딩마감날짜,카테고리,펀딩성공여부,프로젝트명
0,https://www.wadiz.kr/web/campaign/detail/16418,엠케이테크놀로지,스마트폰 사진이 예술작품으로 변하다 : 레볼캠,2018-01-04,2018-02-04,테크·가전,1,레볼캠
1,https://www.wadiz.kr/web/campaign/detail/16272,반디통신기술(주),"일상을 스마트하게, IoT 전등스위치",2018-01-08,2018-02-22,테크·가전,1,WiFi전등스위치
2,https://www.wadiz.kr/web/campaign/detail/14973,(주)스토리포유,"진정한 마음의 평화를 가져다 줄 작은 동반자, ""에스체인""",2018-01-15,2018-02-18,테크·가전,1,에스체인프로
3,https://www.wadiz.kr/web/campaign/detail/16853,(주)토브넷,"""이런 CCTV는 처음이지? 나야 나! 토브넷캠!""",2018-01-19,2018-02-18,테크·가전,1,말하는전구CCTV
4,https://www.wadiz.kr/web/campaign/detail/15982,오정숙,굿바이~ 문콕! 차량용 프리미엄 도어가드 GP-01,2018-01-23,2018-03-07,테크·가전,1,굿바이문콕
5,https://www.wadiz.kr/web/campaign/detail/16617,신텍주식회사,3가지 기술을 하나로! 블루투스 캐치벨!,2018-01-24,2018-03-11,테크·가전,1,위치기반블루투스
6,https://www.wadiz.kr/web/campaign/detail/16213,씨익연구소,"이야기를 전하는 나만의 공기청정기, 에어챗 공기청정기",2018-01-24,2018-03-11,테크·가전,1,에어챗공기청정기
7,https://www.wadiz.kr/web/campaign/detail/14869,(주)야베스 인터내셔널,하이브리드 스마트 워치 'ZeTime 지타임',2018-02-06,2018-03-11,테크·가전,1,스마트워치지타임
8,https://www.wadiz.kr/web/campaign/detail/16950,신지모루,어디에나 딱! 편리하게 부착하는 차량용 거치대 신지모루 볼리에어,2018-02-20,2018-03-18,테크·가전,1,차량용휴대폰거치대
9,https://www.wadiz.kr/web/campaign/detail/16964,골드래빗(주),누구나 원하는 로봇을 만들 수 있습니다. 마운지 로봇!,2018-02-28,2018-04-13,테크·가전,1,놀라운로봇마운지


In [52]:
tech_data2018.to_excel('successful_tech_data2018.xlsx')

In [59]:
# 크롬 드라이버(창 띄우기 않고 데이터 가져오게 설정)
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver', options=options)

for index, row in tech_data2018.iterrows():
    keyword = row.메이커+'+'+row.프로젝트명
    driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+keyword)
    try:
        result_text = driver.find_element_by_css_selector('div.title_desc.all_my span').text.replace(',', '')
        result = re.search(r'(\d+)건', result_text).group(1)
        
    except:
        result = 0
    print(keyword,':', result)
    

엠케이테크놀로지+레볼캠 : 0
반디통신기술(주)+WiFi전등스위치 : 0
(주)스토리포유+에스체인프로 : 0
(주)토브넷+말하는전구CCTV : 0
오정숙+굿바이문콕 : 0
신텍주식회사+위치기반블루투스 : 0
씨익연구소+에어챗공기청정기 : 2
(주)야베스 인터내셔널+스마트워치지타임 : 0
신지모루+차량용휴대폰거치대 : 1
골드래빗(주)+놀라운로봇마운지 : 0
솔라렌코+오클린원음파칫솔 : 0
HK네트웍스+스마트플러그 : 40
지엠엘+시력보호필름 : 0
S-dot(에스다트)+유튜브크리에이터 : 2
스마트드림+스마트버튼팝팝 : 6
GAZE LAB (게이즈랩)+무선충전마우스패드 : 1
아이오티코리아+무선충전거치대 : 8
뉴턴1665 주식회사+기능칫솔콧숨테이프 : 0
에스티에이엠 모바일 네트웍스+지라이더미니선풍기 : 0
주식회사 엣지코프+엣지E6플러스 : 0
ZOOPITER+스마트주차번호패드 : 0
아이로드주식회사+카딱스마트재떨이 : 4
EBIOTECO+방부제제로가글 : 0
(주)데오큐브+탈취항균음쓰레기통 : 0
(주)블루드림워터 / 림피오+몸에좋은물림피오 : 0
(주)오르다+모기퇴치기 : 0
(주)오르다+핸디형미니선풍기 : 0
맥팬코리아+이노블프라이팬 : 0
이온플렉스+수소수텀블러 : 0
먼지몬지+미세먼지측정기 : 4
투유커뮤니케이션즈+위너스멘토 : 7
(주)테크온비젼+화면분할모니터 : 4
주식회사 아이지+환기청정기 : 1
(주)한국전자기술+착붙여쓰는배터리 : 0
솔라렌코+웹카메라커버 : 0
NEOPOP ART+우든티비나무티비 : 0
엘레트론 코리아+타자기키보드 : 6
(주)신지모루+그립감보조신지그립 : 0
아이로드주식회사+카딱스마트충전기 : 4
(주)엔에스디오+최소형오디오 : 0
블루필(주)+미니헤드선풍기 : 0
(주)시선+스마트안경시선비콘 : 4
(주)쿡플레이+스마트주방저울 : 0
(주) 나란+초소형유해물질센서 : 1
원더풀플랫폼+인공지능로봇 : 59
모그피씨+원목미니피씨 : 2
에어러블(Airable)+영유아용공기청정기 : 0
LAB404+골전도헤드폰 : 0
(주